In [ ]:
!pip install openpyxl

In [ ]:
import pandas as pd
import numpy as np

import torch as t
import torch.nn.functional as f

from tqdm.notebook import trange, tqdm

from bs4 import BeautifulSoup

from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer

# Импорт и обработка данных

---

Импортируем данные, избавимся от html тегов и удалим неинформативные сообщения

In [ ]:
data = pd.read_excel('/content/sample_data/dataset_comments.xlsx').drop(['UserSenderId', 'SubmitDate'], axis=1)

In [ ]:
from bs4 import BeautifulSoup

# Удалим html теги
def html_to_text(html):
    soup = BeautifulSoup(html, "html.parser")
    text = soup.get_text(separator=" ")
    text = text.replace("\xa0", " ")
    return text.strip()

data.MessageText = data.MessageText.apply(lambda x: html_to_text(x))

In [ ]:
import re

# Частично избавимся от "шума"
def is_noise(text):
    if len(text) < 3:  # Убираем слишком короткие строки
        return True
    if re.match(r"^[\W\d_]+$", text):  # Только символы и цифры
        return True
    if re.match(r"^[a-zA-Z]+$", text):  # Только латиница
        return True
    return False

data = data[~data.MessageText.apply(is_noise)]

# Аугментация данных

---

Используем предобученную модель для перефразирования предложений, классы для перефразированных предложений сохраняем

Для того, чтобы избавиться от пересечения примеров в будущем, разделим выборку заранее на тренировочную и тестовую

In [ ]:
from sklearn.model_selection import train_test_split

train_data, valid_data = train_test_split(data, test_size=0.2, stratify=data.labels, random_state=42)

In [ ]:
# Посмотрим на балансировку данных
classes_counts = train_data.labels.value_counts()
classes_counts

,count
labels,
1,189
2,149
0,46


In [ ]:
# Теперь посчитаем сколько примеров нужно сгенерировать для каждого класса для балансировки(и умножим на 3)
classes_samples = dict(3 * np.round(classes_counts.max() / classes_counts).astype(int))
classes_samples

{1: 3, 2: 3, 0: 12}

In [ ]:
import torch as t
from transformers import pipeline
from tqdm.notebook import trange, tqdm

paraphrase_model = pipeline(
    "text2text-generation",
    model="cointegrated/rut5-base-paraphraser",
    device=0
)

def paraphrase_text(text, num_return_sequences=3):
    paraphrased = paraphrase_model(
        text,
        max_length=128,
        num_return_sequences=num_return_sequences,
        truncation=True,
        temperature=0.5,
        do_sample=True,
    )

    return [p["generated_text"] for p in paraphrased]

aug_data = {
    'MessageText': [],
    'labels': []
}

for class_, return_seqs in classes_samples.items():
    subdata = train_data[train_data.labels == class_]
    for i in trange(subdata.shape[0]):
        aug_text = paraphrase_text(subdata.iloc[i, 0], return_seqs)

        aug_data['MessageText'] += aug_text
        aug_data['labels'] += [subdata.iloc[i, 1]] * return_seqs

        t.cuda.empty_cache()

aug_data = pd.DataFrame(aug_data)

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Device set to use cpu


  0%|          | 0/189 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


  0%|          | 0/149 [00:00<?, ?it/s]

  0%|          | 0/46 [00:00<?, ?it/s]

In [ ]:
# Объединим данные и избавимся от дубликатов
train_data = pd.concat([train_data, aug_data], axis=0).drop_duplicates()

# Рандомно засэмплируем данные
train_data = train_data.sample(frac=1)

In [ ]:
# Сохраним данные
train_data.to_excel('/content/train_dataset.xlsx')
valid_data.to_excel('/content/valid_dataset.xlsx')